In [13]:
#blingfire text_to_sentence

#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
from pathlib import Path
from blingfire import text_to_sentences

def use_blingfire(text_input):
    wiki_dump_file_in = Path(text_input)
    wiki_dump_file_out = wiki_dump_file_in.parent / \
        f'{wiki_dump_file_in.stem}_preprocessed{wiki_dump_file_in.suffix}'

    print(f'Pre-processing {wiki_dump_file_in} to {wiki_dump_file_out}...')
    with open(wiki_dump_file_out, 'w', encoding='utf-8') as out_f:
        with open(wiki_dump_file_in, 'r', encoding='utf-8') as in_f:
            for line in in_f:
                sentences = text_to_sentences(line)
                out_f.write(sentences + '\n')
    print(f'Successfully pre-processed {wiki_dump_file_in} to {wiki_dump_file_out}...')


if not os.path.exists('enwiki-latest-pages-articles_preprocessed.txt'):
    use_blingfire('enwiki-latest-pages-articles.txt')


In [14]:
!wc -l enwiki-latest-pages-articles_preprocessed.txt


0 enwiki-latest-pages-articles_preprocessed.txt


In [16]:
for linemag in [4, 5, 6]:
    num_lines = 10**linemag
    print(num_lines)
    !head enwiki-latest-pages-articles_preprocessed.txt -n {num_lines} > sample{num_lines}.txt

10000
head: -n: No such file or directory
head: 10000: No such file or directory
100000
head: -n: No such file or directory
head: 100000: No such file or directory
1000000
head: -n: No such file or directory
head: 1000000: No such file or directory


In [8]:

!ls -l

total 72
-rw-r--r--  1 wojm  staff   210 Aug 18 21:27 README.md
-rw-r--r--  1 wojm  staff   430 Aug 18 21:31 download_wiki_dump.sh
-rw-r--r--  1 wojm  staff     0 Aug 18 21:31 enwiki-latest-pages-articles_preprocessed.txt
-rw-r--r--  1 wojm  staff   663 Aug 18 21:31 extract_and_clean_wiki_dump.sh
-rw-r--r--  1 wojm  staff  8809 Aug 18 21:30 prepare_data.ipynb
-rw-r--r--  1 wojm  staff    54 Aug 18 21:31 sample10000.txt
-rw-r--r--  1 wojm  staff    54 Aug 18 21:31 sample100000.txt
-rw-r--r--  1 wojm  staff    54 Aug 18 21:31 sample1000000.txt
